In [ ]:
%load_ext autoreload
%autoreload 2
# Copyright (c) Meta Platforms, Inc. and affiliates.
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.
import argparse
import dataclasses
import sys
import timeit
from typing import Tuple

import click
import matplotlib.pyplot as plt
import numpy as np
import open3d
import torch
from tqdm import tqdm

# from home_robot.mapping.voxel import SparseVoxelMap
from home_robot.utils.point_cloud_torch import unproject_masked_depth_to_xyz_coordinates


In [ ]:
from pathlib import Path
import gzip
import pickle
result_path = Path('/private/home/ssax/concept-graphs/conceptgraph/cached_eqa/117-scannet-scene0277_00/pcd_saves/full_pcd_ram_withbg_allclasses_overlap_maskconf0.25_simsum1.2_dbscan.1.pkl.gz')
result_path = Path('/private/home/ssax/concept-graphs/conceptgraph/cached_eqa/069-hm3d-HMkoS756sz6/pcd_saves/full_pcd_ram_withbg_allclasses_overlap_maskconf0.25_simsum1.2_dbscan.1.pkl.gz')
with gzip.open(result_path, "rb") as f:
    results = pickle.load(f)
    
# Add way to show post-nms boxes

In [ ]:
from home_robot.mapping.instance.core import Instance, InstanceView
from home_robot.utils.point_cloud_torch import get_bounds
def instance_from_dict(instance_dict):
    # Create an Instance object from the given dictionary.
    instance = Instance()
    instance.name = instance_dict.get('class_name')[0]
    instance.category_id = torch.LongTensor([instance_dict.get('class_id')[0]]).squeeze()
    instance.point_cloud = torch.tensor(instance_dict.get('pcd_np')) if instance_dict.get('pcd_np') is not None else None
    instance.point_cloud_rgb = torch.tensor(instance_dict.get('pcd_color_np')) if instance_dict.get('pcd_color_np') is not None else None
    # Assuming 'clip_ft' or 'text_ft' can be used as point cloud features.
    instance.point_cloud_features = torch.tensor(instance_dict.get('clip_ft')) if instance_dict.get('clip_ft') is not None else None
    instance.bounds = get_bounds(instance.point_cloud) if instance_dict.get('bbox_np') is not None else None
    # Assuming 'conf' is the confidence score
    instance.score = instance_dict.get('conf')
    return instance

# Assuming you have a dictionary 'instance_dict' with the provided keys
# instance_dict = {...}

# Convert the dictionary to an Instance object


In [ ]:
objects = [instance_from_dict(objdict) for objdict in results['objects'] if objdict['pcd_np'].shape[0] > 10]
bg_objects = [instance_from_dict(objdict) for objdict in results['bg_objects'] if objdict['pcd_np'].shape[0] > 10]

In [ ]:
from home_robot.mapping.voxel.voxel import SparseVoxelMap
from home_robot.utils.voxel import VoxelizedPointcloud
svm = SparseVoxelMap()
for obj in objects + bg_objects:
    svm.voxel_pcd.add(
        points = obj.point_cloud,
        rgb = obj.point_cloud_rgb,
        features = None,
    )
for i, obj in enumerate(objects):
    svm.instances.instances[0][i] = obj
    
    
    

In [ ]:
bounds, names = zip(
         *[(v.bounds, v.category_id) for v in svm.get_instances()]
)

In [ ]:
svm.show(backend='pytorch3d', boxes_plot_together=False,  height=1000,)

In [ ]:
instance_object._show_point_cloud_pytorch3d()

In [ ]:
.keys()

In [ ]:
results['objects'][1]['pcd_np']

In [ ]:
from home_robot.datasets.scannet import ScanNetDataset, ReferIt3dDataConfig, ScanReferDataConfig
data = ScanNetDataset(
    root_dir = '/private/home/ssax/home-robot/src/home_robot/home_robot/datasets/scannet/data',
    frame_skip = 180,
    n_classes = 50,
    referit3d_config = ReferIt3dDataConfig(),
    scanrefer_config = ScanReferDataConfig(),
)

# Load specific scene
# scene0192_00 -- small scene
# 'scene0000_00' -- large scene
idx = data.scene_list.index("scene0192_00") #'scene0000_00'
# idx = 0
print(f"Loaded images of (h: {data.height}, w: {data.width}) - resized from ({data.DEFAULT_HEIGHT},{data.DEFAULT_WIDTH})")
scene_obs = data.__getitem__(idx, show_progress=True)

# Load GT mesh
from pytorch3d.io import IO, load_obj, load_ply
from pytorch3d.structures import Pointclouds
def transform_points(points, transform_mat):
    return (torch.cat([points, torch.ones_like(points[:,:1])], dim=-1) @ transform_mat.T)[...,:3]

scene_id = scene_obs['scan_name']
print("Loading GT mesh for", scene_id)
pointcloud = IO().load_pointcloud(data.root_dir / f'scans/{scene_id}/{scene_id}_vh_clean.ply')
verts = transform_points(pointcloud.points_packed(), scene_obs['axis_align_mats'][0])
pointcloud_aligned =  Pointclouds(points = [verts], features = [pointcloud.features_packed()])
print(f"GT mesh: {len(verts)} verts")

# Create Voxel + Instances map
Not working? Try building up to the full SparseVoxelMap by running the "Debugging" subsections below this set of cells

In [ ]:
from home_robot.mapping.voxel import SparseVoxelMap
from home_robot.perception.detection.detic.detic_perception import DeticPerception
from home_robot.perception.constants import RearrangeDETICCategories
from home_robot.utils.config import get_config

from home_robot.perception.wrapper import (
    OvmmPerception,
    build_vocab_from_category_map,
)

from enum import IntEnum, auto

class SemanticVocab(IntEnum):
    FULL = auto()
    SIMPLE = auto()
    ALL = auto()

config = get_config('configs/model/semantic_sensor/grounded_sam_perception_home_robot.yaml')[0]

segmenter = OvmmPerception(
    config=config,
    gpu_device_id=0,
)
simple_vocab = RearrangeDETICCategories(dict(zip(
    data.METAINFO['CLASS_IDS'], # IDs [1, 3, 4, 5, ..., 65]
    data.METAINFO['CLASS_NAMES'] # [wall, floor, cabinet, ...]
)))

segmenter.update_vocabulary_list(simple_vocab, SemanticVocab.SIMPLE)
segmenter.set_vocabulary(SemanticVocab.SIMPLE)


In [ ]:
# Get detections and move to CUDA
segmenter._segmentation.box_threshold = 0.25
segmenter._segmentation.text_threshold = 0.25

from home_robot.core.interfaces import Observations
import dataclasses
scene_obs['instance_map'] = []
scene_obs['instance_classes'] = []
scene_obs['instance_scores'] = []
device = 'cuda'
with torch.no_grad():
    # Ignore_classes
    instance_map, instance_classes, instance_scores = [], [], []
    semantic_frames = [
        
    ]
    semantic_map = []
    
    for im in scene_obs['images'].cpu().numpy():
        obs = Observations(rgb = (im * 255).astype(np.uint8), gps=None, compass=None, depth=None)
        res = segmenter.predict(obs)
        semantic_map.append(torch.from_numpy(res.semantic).int().to(device))
        res = res.task_observations
        instance_map.append(torch.from_numpy(res['instance_map']).int().to(device))
        instance_classes.append(torch.from_numpy(res['instance_classes']).int().to(device))
        instance_scores.append(torch.from_numpy(res['instance_scores']).float().to(device))
        semantic_frame = res['semantic_frame'] if res['semantic_frame'] is not None else res['instance_map']
        semantic_frames.append(torch.from_numpy(semantic_frame))
    scene_obs['instance_map'] = torch.stack(instance_map, dim=0)
    scene_obs['instance_classes'] = instance_classes
    scene_obs['instance_scores'] = instance_scores
    scene_obs['images'] = scene_obs['images'].to(device)
    scene_obs['depths'] = scene_obs['depths'].to(device)
    scene_obs['intrinsics'] = scene_obs['intrinsics'].to(device)
    scene_obs['poses'] = scene_obs['poses'].to(device)
    scene_obs['semantic_frame'] = semantic_frames

# Some inforamtion about the last image
print("Information about the results:")
for k, v in res.items():
    if v is not None:
        print(k, v.shape, v.min(), v.max(), len(np.unique(v)))

# for sf, sm, d in zip(scene_obs['semantic_frame'], semantic_map, scene_obs['depths']):
#     sf, sm, d = sf.cpu(), sm.cpu(), d.cpu()
#     f, axarr = plt.subplots(1, 3)  # n_views rows, 1 col
#     axarr[0].imshow(sf)
#     axarr[1].imshow(sm)
#     axarr[2].imshow(sf * ((0.1 < d) & (d < 4.0)).unsqueeze(-1))
#     plt.show()

In [ ]:
plt.imshow((semantic_map[-1].cpu().numpy()).astype(np.uint8))


In [ ]:
# plt.imshow((scene_obs['images'].cpu().numpy()[-1] * 255).astype(np.uint8))
plt.imshow((scene_obs['instance_map'].cpu().numpy()[-1]).astype(np.uint8))


In [ ]:
from home_robot.perception.encoders import ClipEncoder
encoder = ClipEncoder(version="ViT-B/32", device = 'cuda:1')

In [ ]:
# encoded_classes = [encoder.encode_text]

In [ ]:
from home_robot.mapping.instance.instance_map import ViewMatchingConfig
from home_robot.mapping.instance.matching import Bbox3dOverlapMethodEnum
# Create SparseVoxelMap from observations
svm = SparseVoxelMap(
    background_instance_label=-1,
    instance_memory_kwargs = dict(
        erode_mask_num_pix=5,
        erode_mask_num_iter=5,
        # iou_threshold=0.3,
        # instance_association='bbox_one_sided_iou'
        view_matching_config = ViewMatchingConfig(
            within_class = False,
            box_match_mode = Bbox3dOverlapMethodEnum.ONE_SIDED_IOU,
            box_min_iou_thresh = 0.6,
            box_overlap_weight = 0.3,
            visual_similarity_weight = 1.0
        ) 
    ),
    encoder = encoder
    # min_iou=0.25
)

n_frames = len(scene_obs['images'])
with torch.no_grad():
    for i in tqdm(range(n_frames)):
        svm.add(
            rgb = scene_obs['images'][i],
            depth = scene_obs['depths'][i].squeeze(-1),
            feats = scene_obs['images'][i],
            camera_K = scene_obs['intrinsics'][i][:3,:3],
            camera_pose = scene_obs['poses'][i], #scene_obs['axis_align_mats'][i] @ scene_obs['poses'][i],
            instance_image = scene_obs['instance_map'][i] ,
            instance_classes = scene_obs['instance_classes'][i],
            instance_scores = scene_obs['instance_scores'][i],
            # obs_info = extra information about this timestep
            # instance_info = list of length num_instances
        )
print(f"Pre-NMS found {len(svm.get_instances())} global instances")

svm.instances.global_instance_nms(0, within_category=False, nms_iou_thresh=0.3)
print(f"Found {len(svm.get_instances())} global instances")

In [ ]:
svm.show(backend='pytorch3d',
    height=1000,
    boxes_plot_together=False,
    boxes_name_int_to_display_name_dict=segmenter.seg_id_to_name
)

In [ ]:
for bounds in zip(gt_bounds):
    torch.set_printoptions(sci_mode=False)
    print(bounds)

In [ ]:
from evaluation.obj_det import eval_bboxes_and_print

# Predicted
pred_bounds = torch.stack([inst.bounds.cpu() for inst in svm.get_instances()])
pred_class = torch.stack([inst.category_id.cpu() for inst in svm.get_instances()])
pred_scores = torch.stack([torch.max(torch.stack([v.score for v in ins.instance_views])).cpu() for ins in svm.get_instances()])

# GT
gt_bounds = scene_obs['boxes_aligned']
# Map GT class labels to match the segmenter since the segmenter might have extra classes (i.e. 'background, other')
index = torch.tensor(list(segmenter.vocabulary_id_to_name.keys()), dtype=torch.int64)
src = torch.tensor([segmenter.name_to_seg_id[name] for name in segmenter.vocabulary_id_to_name.values()], dtype=torch.int64)
data_idx_to_seg_idx = torch.zeros(max(segmenter.vocabulary_id_to_name.keys()) + 1, dtype=torch.int64)
data_idx_to_seg_idx.scatter_(dim=0, index=index, src=src).int()
gt_class = data_idx_to_seg_idx[scene_obs['box_classes']]


_ = eval_bboxes_and_print(
    box_gt_bounds = [gt_bounds],
    box_gt_class = [gt_class],
    box_pred_bounds = [pred_bounds],
    box_pred_class = [pred_class],
    box_pred_scores = [pred_scores],
    match_within_class = True,
    iou_thr = (0.25, 0.5, 0.75),
    label_to_cat = segmenter.seg_id_to_name 
)

In [ ]:
scene_obs.keys()

In [ ]:
from evaluation.refer_exp import eval_obj_selection_bboxes

# Predicted
pred_bounds = torch.stack([inst.bounds.cpu() for inst in svm.get_instances()])
pred_class = torch.stack([inst.category_id.cpu() for inst in svm.get_instances()])
pred_scores = torch.stack([torch.max(torch.stack([v.score for v in ins.instance_views])).cpu() for ins in svm.get_instances()])

# GT
gt_bounds = scene_obs['boxes_aligned']
# Map GT class labels to match the segmenter since the segmenter might have extra classes (i.e. 'background, other')
index = torch.tensor(list(segmenter.vocabulary_id_to_name.keys()), dtype=torch.int64)
src = torch.tensor([segmenter.name_to_seg_id[name] for name in segmenter.vocabulary_id_to_name.values()], dtype=torch.int64)
data_idx_to_seg_idx = torch.zeros(max(segmenter.vocabulary_id_to_name.keys()) + 1, dtype=torch.int64)
data_idx_to_seg_idx.scatter_(dim=0, index=index, src=src).int()
gt_class = data_idx_to_seg_idx[scene_obs['box_classes']]


_ = eval_obj_selection_bboxes(
    box_gt_bounds = [gt_bounds],
    box_gt_class = [gt_class],
    box_gt_ids = [],
    exp_target_ids = [],
    box_pred_bounds = [pred_bounds],
    iou_thr = (0.25, 0.5, 0.75),
    # label_to_cat = segmenter.seg_id_to_name 
)

In [ ]:
# Visualize an instance
idx = [int(c) for c in classes].index(0)

instance = svm.get_instances()[idx]
fig = instance._show_point_cloud_pytorch3d()
fig.show()
_ = instance._show_instance_view_frames()
# plt.imshow(scene_obs['semantic_frame'][n_frames - 1])

In [ ]:
from eval.utils import eval_instance_bbox_detection
results = eval_instance_bbox_detection(
    
)
def eval_bboxes_and_print(
    box_gt_bounds: Tensor,
    box_gt_class: Tensor,
    box_pred_bounds: Tensor,
    box_pred_class: Tensor,
    box_pred_scores: Tensor,
    match_within_class: bool = True,
    iou_thr: Sequence[float] = (0.25, 0.5, 0.75)
):

## Debugging: Build VoxelizedPointcloud

In [ ]:
# Build voxelized pointcloud from GT points

from torch_geometric.nn.pool.voxel_grid import voxel_grid
from torch_geometric.nn.pool.consecutive import consecutive_cluster
from torch_geometric.utils import add_self_loops, scatter
import time

from home_robot.utils.voxel import (
    VoxelizedPointcloud,
    voxelize, 
    reduce_pointcloud
)

# Move to CUDA 
points = pointcloud_aligned.points_packed().cuda()
features = pointcloud_aligned.features_packed().cuda()
device = 'cuda:0'
for key in ['images', 'depths', 'intrinsics', 'poses']:
    scene_obs[key] = scene_obs[key].to(device)

start = time.time()
with torch.no_grad():
    cluster_idx, cluster_consecutive_idx, _ = voxelize(points, 0.05)
    points_reduced, feats_reduced, weights_reduced, rgb_cluster = reduce_pointcloud(cluster_consecutive_idx, points, features, rgbs=features)
 
end = time.time()
print(end - start)


In [ ]:
# Build voxelized pointcloud from observations

from pytorch3d.structures import Pointclouds as p3dPointclouds
from pytorch3d.vis.plotly_vis import AxisArgs

from home_robot.utils.bboxes_3d_plotly import plot_scene_with_bboxes
from home_robot.utils.voxel import VoxelizedPointcloud

# Build voxel map
min_depth = 0.1
max_depth = 4.0
with torch.no_grad():
    vpc = VoxelizedPointcloud()

    for i in tqdm(range(len(scene_obs['images']))):
        rgb = scene_obs['images'][i]
        depth = scene_obs['depths'][i].squeeze(-1)
        camera_pose = scene_obs['poses'][i]
        camera_K = scene_obs['intrinsics'][i]
        
        full_world_xyz = unproject_masked_depth_to_xyz_coordinates( # Batchable!
            depth=depth.unsqueeze(0).unsqueeze(1),
            pose=camera_pose.unsqueeze(0),  
            inv_intrinsics=torch.linalg.inv(camera_K[:3, :3]).unsqueeze(0),
        )

        valid_depth = torch.full_like(rgb[:,0], fill_value=True, dtype=torch.bool)
        if depth is not None:
            valid_depth = (depth > min_depth) & (depth < max_depth)
        
        rgb = rgb[valid_depth].reshape(-1, 3)
        world_xyz = full_world_xyz[valid_depth.flatten()]
        
        vpc.add(world_xyz, features=rgb, rgb=rgb)

points_reduced, features_reduced, weights_reduced, rgb_reduced = vpc.get_pointcloud()

# Visualize
ptc = p3dPointclouds(
    points = [points_reduced.detach().cpu()],
    features = [rgb_reduced.detach().cpu()],
)
fig = plot_scene_with_bboxes(
    plots = { f"{scene_id}": { 
                                "Points": ptc,
                                "GT points": pointcloud_aligned,

                            }
    },
    xaxis={"backgroundcolor":"rgb(200, 200, 230)"},
    yaxis={"backgroundcolor":"rgb(230, 200, 200)"},
    zaxis={"backgroundcolor":"rgb(200, 230, 200)"}, 
    axis_args=AxisArgs(showgrid=True),
    pointcloud_marker_size=3,
    pointcloud_max_points=200_000,
    boxes_wireframe_width=3,
    boxes_add_cross_face_bars=False,
    boxes_name_int_to_display_name_dict = dict(zip([int(i) for i in data.METAINFO['seg_valid_class_ids']], data.METAINFO['classes'])),
    boxes_plot_together=True,
    height=1000,
    # width=1000,
)
fig

## Experimental: Conceptfusion

In [ ]:
from gradslam.slam.pointfusion import PointFusion
from gradslam.structures.pointclouds import Pointclouds as gsPointclouds
from gradslam.structures.rgbdimages import RGBDImages

device = 'cuda:0'
slam = PointFusion(odom="icp", dsratio=2, dist_thresh=0.01, device=device, use_embeddings=True)
for key in ['images', 'depths', 'intrinsics', 'poses']:
    scene_obs[key] = scene_obs[key].to(device)

In [ ]:
frame_cur, frame_prev = None, None
pointclouds = gsPointclouds(
    device=device,
)
with torch.no_grad():
    for i in tqdm(range(len(scene_obs['images']))):
        frame_cur = RGBDImages(
                scene_obs['images'][i].unsqueeze(0).unsqueeze(0),
                scene_obs['depths'][i].unsqueeze(0).unsqueeze(0),
                scene_obs['intrinsics'][i].unsqueeze(0).unsqueeze(0),
                scene_obs['poses'][i].unsqueeze(0).unsqueeze(0),
                embeddings=scene_obs['images'][i].unsqueeze(0).unsqueeze(0).half(),
            )
        pointclouds, _ = slam.step(pointclouds, frame_cur, frame_prev) # last arg is frame_prev but not used with odom=gt
        frame_prev = frame_cur

        # # Ignore_classes
        # instance_map, instance_classes, instance_scores = [], [], []
        # for im in scene_obs['images'].numpy():
        #     obs = Observations(rgb = im * 255, gps=None, compass=None, depth=None)
        #     res = segmenter.predict(obs).task_observations
        #     instance_map.append(torch.from_numpy(res['instance_map']).int())
        #     instance_classes.append(torch.from_numpy(res['instance_classes']).int())
        #     instance_scores.append(torch.from_numpy(res['instance_scores']).float())
        # scene_obs['instance_map'] = instance_map
        # scene_obs['instance_classes'] = instance_classes
        # scene_obs['instance_scores'] = instance_scores    
        #     frame_cur = RGBDImages(
        #         _color.unsqueeze(0).unsqueeze(0),
        #         _depth.unsqueeze(0).unsqueeze(0),
        #         intrinsics.unsqueeze(0).unsqueeze(0),
        #         _pose.unsqueeze(0).unsqueeze(0),
        #         embeddings=_embedding.unsqueeze(0).unsqueeze(0),
        #     )
        #     pointclouds, _ = slam.step(pointclouds, frame_cur, frame_prev)
        #     # frame_prev = frame_cur
        #     torch.cuda.empty_cache()

In [ ]:
from home_robot.utils.bboxes_3d_plotly import plot_scene_with_bboxes
from pytorch3d.structures import Pointclouds as p3dPointclouds
from pytorch3d.vis.plotly_vis import AxisArgs

ptc = p3dPointclouds(
    points = pointclouds.points_list,
    features = pointclouds.embeddings_padded,
)
fig = plot_scene_with_bboxes(
    plots = { f"{scene_id}": { 
                                "Points": ptc,
                                # # "Boxes": join_boxes_as_scene(svm.instance_bboxes3d),
                                # "All boxes": global_boxes,
                                # "Global boxes": global_boxes,
                                # "GT boxes": gt_boxes,
                                "GT points": p3dPointclouds(points=[aligned_verts[:, :3]]),
                                # "cameras": cameras,
                            }
    },
    xaxis={"backgroundcolor":"rgb(200, 200, 230)"},
    yaxis={"backgroundcolor":"rgb(230, 200, 200)"},
    zaxis={"backgroundcolor":"rgb(200, 230, 200)"}, 
    axis_args=AxisArgs(showgrid=True),
    pointcloud_marker_size=3,
    pointcloud_max_points=30_000,
    boxes_wireframe_width=3,
    boxes_add_cross_face_bars=False,
    boxes_name_int_to_display_name_dict = dict(zip([int(i) for i in data.METAINFO['seg_valid_class_ids']], data.METAINFO['classes'])),
    boxes_plot_together=True,
    height=1000,
    # width=1000,
)
fig

In [ ]:
# K = scene_obs['intrinsics'][0][:3,:3]
# depth = scene_obs['depths'][0].squeeze().unsqueeze(0).unsqueeze(1)
# valid_depth  = (0.1 < depth) & (depth < 4.0)

# xyz = unproject_masked_depth_to_xyz_coordinates(
#     depth = depth,
#     mask  = ~valid_depth,
#     pose  = torch.eye(4).unsqueeze(0),
#     inv_intrinsics = torch.linalg.inv(K).unsqueeze(0),
# )
# rgb = scene_obs['images'][0].reshape(-1,3)[valid_depth.flatten()]
# print(scene_obs['image_paths'][0])
# print(f"Proportion depth valid: {float(valid_depth.float().mean())}")
# print(f"Depth min + max: {float(depth.flatten()[valid_depth.flatten()].min())}, {float(depth.flatten()[valid_depth.flatten()].max())}")
# print("These are the mins-maxes along each world axis. They should be in meters:")
# for i in range(3):
#     print(f"  {i}: ({float(xyz[:,i].min())}, {float(xyz[:,i].max())})")

In [ ]:
plt.imshow(res['semantic_frame'])
plt.show()
plt.imshow(scene_obs['instance_map'][-1] == 0)
plt.show()

In [ ]:
# # Create SparseVoxelMap from observations
# from home_robot.mapping.voxel import SparseVoxelMap
# svm = SparseVoxelMap()
# with torch.no_grad():
#     for i in tqdm(range(len(scene_obs['images']))):
        
#         # Unproject and add to SparseVoxelMap
#         K = scene_obs['intrinsics'][i][:3,:3]
#         depth = scene_obs['depths'][i].squeeze().unsqueeze(0).unsqueeze(1)
#         valid_depth  = (0.1 < depth) & (depth < 4.0)

#         xyz = unproject_masked_depth_to_xyz_coordinates(
#             depth = depth,
#             mask  = ~valid_depth,
#             pose  = (scene_obs['axis_align_mats'][i] @ scene_obs['poses'][i]).unsqueeze(0), #torch.eye(4).unsqueeze(0),
#             inv_intrinsics = torch.linalg.inv(K).unsqueeze(0),
#         )
#         rgb = scene_obs['images'][i].reshape(-1,3)[valid_depth.flatten()]
#         svm.add(
#             world_xyz = xyz,
#             feats = rgb,
#         )

In [ ]:
# Create SparseVoxelMap from observations
from home_robot.mapping.voxel import SparseVoxelMap, SparseVoxelMapWithInstances
svm = SparseVoxelMapWithInstances(background_instance_id=-1, min_iou=0.25)
with torch.no_grad():
    for i in tqdm(range(len(scene_obs['images']))):
        svm.add(
            rgb = scene_obs['images'][i],
            depth = scene_obs['depths'][i].squeeze(-1),
            feats = scene_obs['images'][i],
            cam_K = scene_obs['intrinsics'][i][:3,:3],
            cam_to_world = scene_obs['axis_align_mats'][i] @ scene_obs['poses'][i],
            instance_masks = scene_obs['instance_map'][i] ,
            instance_classes = scene_obs['instance_classes'][i],
            instance_scores = scene_obs['instance_scores'][i],
            # obs_info = extra information about this timestep
            # instance_info = list of length num_instances
        )


In [ ]:
# -> SparseVoxelMapWithInstanceViews.show(backend='pytorch3d')

# Plot GT scene
from home_robot.utils.bboxes_3d import BBoxes3D, join_boxes_as_scene, join_boxes_as_batch
from home_robot.utils.bboxes_3d_plotly import plot_scene_with_bboxes
from pytorch3d.vis.plotly_vis import AxisArgs
from pytorch3d.structures import Pointclouds
import seaborn as sns

colors = torch.tensor(sns.color_palette("husl", len(scene_obs['boxes_aligned'])))
gt_boxes = BBoxes3D(
    bounds = [scene_obs['boxes_aligned']],
    # features = [colors[0].unsqueeze(0).expand(27,3)],
    features = [colors],
    names = [scene_obs['box_classes'].unsqueeze(-1)]
)
# # detected_boxes = BBoxes3D(
# #     bounds = [torch.stack([v.bounds_3d for v in inst]) for inst in svm.instance_views],
# #     # features = [colors],
# #     names = [torch.stack([v.category_id for v in inst]).unsqueeze(-1) for inst in svm.instance_views]
# # )
detected_boxes = BBoxes3D(
    bounds = [torch.cat([torch.stack([v.bounds_3d for v in inst]) for inst in svm.instance_views], dim=0)],
    # features = [colors],
    names = [torch.cat([torch.stack([v.category_id for v in inst]).unsqueeze(-1) for inst in svm.instance_views], dim=0)],
)
global_boxes = BBoxes3D(
    bounds = [svm.global_bbox_bounds],
    # features = [colors],
)

fig = plot_scene_with_bboxes(
    plots = { f"{scene_id}": { 
                                "Points": svm.global_voxel_grid._pcl,
                                # "Boxes": join_boxes_as_scene(svm.instance_bboxes3d),
                                "All boxes": global_boxes,
                                "Global boxes": global_boxes,
                                "GT boxes": gt_boxes,
                                "GT points": Pointclouds(points=[aligned_verts[:, :3]]),
                                # "cameras": cameras,
                            }
    },
    xaxis={"backgroundcolor":"rgb(200, 200, 230)"},
    yaxis={"backgroundcolor":"rgb(230, 200, 200)"},
    zaxis={"backgroundcolor":"rgb(200, 230, 200)"}, 
    axis_args=AxisArgs(showgrid=True),
    pointcloud_marker_size=3,
    pointcloud_max_points=30_000,
    boxes_wireframe_width=3,
    boxes_add_cross_face_bars=False,
    boxes_name_int_to_display_name_dict = dict(zip([int(i) for i in data.METAINFO['seg_valid_class_ids']], data.METAINFO['classes'])),
    boxes_plot_together=True,
    height=1000,
    # width=1000,
)
fig

In [ ]:
# -> SparseVoxelMapWithInstanceViews.show(backend='pytorch3d')

# Plot GT scene
from home_robot.utils.bboxes_3d import BBoxes3D
from home_robot.utils.bboxes_3d_plotly import plot_scene_with_bboxes
from pytorch3d.vis.plotly_vis import AxisArgs
from pytorch3d.structures import Pointclouds
import seaborn as sns

# colors = torch.tensor(sns.color_palette("husl", len(scene_obs['boxes_aligned'])))
# boxes = BBoxes3D(
#     bounds = [scene_obs['boxes_aligned']],
#     # features = [colors[0].unsqueeze(0).expand(27,3)],
#     features = [colors],
#     names = [scene_obs['box_classes'].unsqueeze(-1)]
# )
fig = plot_scene_with_bboxes(
    plots = { f"{scene_id}": { 
                                # "Points": svm.global_voxel_grid._pcl, # svm._pcl,
                                "Boxes": gt_boxes,
                                "GT points": Pointclouds(points=[aligned_verts[:, :3]]),
                                # "cameras": cameras,
                            }
    },
    xaxis={"backgroundcolor":"rgb(200, 200, 230)"},
    yaxis={"backgroundcolor":"rgb(230, 200, 200)"},
    zaxis={"backgroundcolor":"rgb(200, 230, 200)"}, 
    axis_args=AxisArgs(showgrid=True),
    pointcloud_marker_size=3,
    pointcloud_max_points=30_000,
    boxes_wireframe_width=3,
    boxes_add_cross_face_bars=False,
    boxes_name_int_to_display_name_dict = dict(zip([int(i) for i in data.METAINFO['seg_valid_class_ids']], data.METAINFO['classes'])),
    height=1000,
    # width=1000,
)
fig

In [ ]:
import open3d
from home_robot.utils.point_cloud import numpy_to_pcd, pcd_to_numpy, show_point_cloud, get_xyz_coordinates
from home_robot.mapping.voxel import SparseVoxelMap

# class ObjectPointcloudRepresentation(object):
#     def __init__(self, visualize_planner=False):
#         self.started = False
#         self.voxel_map = SparseVoxelMap(resolution=0.01)

#     def step(self, rgb, depth, cam_to_world, K, visualize_map=False):
#         """Step the collector. Get a single observation of the world. Remove bad points, such as
#         those from too far or too near the camera."""
#         # Get the camera pose and make sure this works properly
#         camera_pose = cam_to_world
#         # Get RGB and depth as necessary
#         orig_rgb = rgb.clone().detach()
#         orig_depth = depth.clone().detach()

#         assert rgb.shape[:-1] == depth.squeeze(-1).shape, f"{rgb.shape} {depth.shape}"
#         assert rgb.ndim == 3
#         height, width, _ = rgb.shape
        
#         K = K[:3,:3]
#         depth = depth.squeeze().unsqueeze(0).unsqueeze(1)
#         valid_depth  = (0.1 < depth) & (depth < 4.0)
#         xyz = get_xyz_coordinates(
#             depth = depth,
#             mask  = ~valid_depth,
#             pose  = torch.eye(4).unsqueeze(0),
#             inv_intrinsics = torch.linalg.inv(K).unsqueeze(0),
#         ) 
        
#         # apply depth filter
#         depth = depth.reshape(-1)
#         rgb = rgb.reshape(-1, 3)
#         xyz = xyz.reshape(-1, 3)
#         rgb = rgb[valid_depth.flatten()]
#         # xyz = xyz[valid_depth.flatten()]
#         # TODO: remove debug code
#         # For now you can use this to visualize a single frame
#         # show_point_cloud(xyz, rgb / 255, orig=np.zeros(3))
#         self.voxel_map.add(
#             camera_pose.numpy(),
#             xyz.numpy(),
#             rgb.numpy(),
#             depth=depth.numpy(),
#             K=K.numpy(),
#             orig_rgb=orig_rgb.numpy(),
#             orig_depth=orig_depth.numpy(),
#         )
#         if visualize_map:
#             self.voxel_map.get_2d_map(debug=True)

#     def get_2d_map(self):
#         """Get 2d obstacle map for low level motion planning and frontier-based exploration"""
#         return self.voxel_map.get_2d_map()

#     def show(self) -> Tuple[np.ndarray, np.ndarray]:
#         """Display the aggregated point cloud."""

#         # Create a combined point cloud
#         # Do the other stuff we need
#         pc_xyz, pc_rgb = self.voxel_map.get_data()
#         show_point_cloud(pc_xyz, pc_rgb / 255, orig=np.zeros(3))
#         return pc_xyz, pc_rgb

import collections.abc    
from typing import Optional, Union

from home_robot.utils.point_cloud import numpy_to_pcd, pcd_to_numpy, show_point_cloud, get_xyz_coordinates
from home_robot.mapping.voxel import SparseVoxelMap
# Recursive d.update(u). Good for configs!
def update(d, u):
    # https://stackoverflow.com/questions/3232943/update-value-of-a-nested-dictionary-of-varying-depth
    for k, v in u.items():
        if isinstance(v, collections.abc.Mapping):
            d[k] = update(d.get(k, {}), v)
        else:
            d[k] = v
    return d

class ObjectPointcloudRepresentation:
"""Simple class to collect RGB, Depth, and Pose information for building 3d spatial-semantic
maps for the robot. Needs to subscribe to:
- color images
- depth images
- camera info
- joint states/head camera pose
- base pose (relative to world frame)

This is an example collecting the data; not necessarily the way you should do it.
"""

# TODO(sasha): Remove dependence on robot and semantic_sensor.
#   instead, step() is an interface with a TYPE of output.
def __init__(self,
        robot # obs = robot.get_observation(),
        #  obs.rgb
        #  obs.depth
        #  obs.xyz (depth unprojected to camera coords) # potential convention mismatch
        #  obs.camera_pose
        semantic_sensor=None,
        #  obs = semantic_sensor.predict(rgb, depth, xyz, camera_pose)
        #  self.observations.append((camera_pose, xyz, feats, base_pose, obs, info))
        #   world_xyz = trimesh.transform_points(xyz, camera_pose)
        #   instances = InstanceView.create_from_observations(
        #       world_xyz, valid_depth, obs, self._seq
        #   )
        visualize_planner=False
    ):
    self.robot = robot  # Get the connection to the ROS environment via agent
    # Run detection here
    self.semantic_sensor = semantic_sensor #
    self.started = False
    self.robot_model = HelloStretchKinematics(visualize=visualize_planner)
    self.voxel_map = SparseVoxelMap(resolution=0.01)

def step(
        self,
        rgb: torch.Tensor, 
        depth: torch.Tensor,
        camera_pose: torch.Tensor, 
        base_pose_plane: Optional[torch.Tensor] ,
        instance_bounds: Optional[torch.Tensor],
        xyz: Optional[torch.Tensor] = None, #points in camera coordinates -- but we never use these and we filter xyz based on depth. 
        # Why not just remove the dependence, since it's can be tricky for people to create the XYZs themselves.
        # Then we can avoid passing around the xyz everywhere, and we can backprop into depth if 
        visualize_map=False
    ):
    """
        Step the collector. Get a single observation of the world. Remove bad points, such as
        those from too far or too near the camera.
        
        rgb: [h, w, 3]
        depth: [h, w]
        camera_pose: [4, 4] cam_to_world using openCV coordinates
        base_pose_plane: Optional [x,y,theta] of the base position + orientation
        instance_bounds: [K, 3, 2] of bounding boxes mins + maxes along each axis
        
    """
    # obs = self.robot.get_observation()

    # rgb = obs.rgb
    # depth = obs.depth
    # xyz = obs.xyz
    # camera_pose = obs.camera_pose
    base_pose = np.array([obs.gps[0], obs.gps[1], obs.compass[0]])

    # Get RGB and depth as necessary
    orig_rgb = rgb.clone().detach()
    orig_depth = depth.clone().detach()
    assert rgb.shape[:-1] == depth.squeeze(-1).shape, f"{rgb.shape} {depth.shape}"
    assert rgb.ndim == 3
    height, width, _ = rgb.shape
    
    K = K[:3,:3]
    depth = depth.squeeze().unsqueeze(0).unsqueeze(1)
    valid_depth  = (0.1 < depth) & (depth < 4.0)
    xyz = get_xyz_coordinates(
        depth = depth,
        mask  = ~valid_depth,
        pose  = torch.eye(4).unsqueeze(0),
        inv_intrinsics = torch.linalg.inv(K).unsqueeze(0),
    ) 
    
    # Semantic prediction
    obs = self.semantic_sensor.predict(obs)

    # TODO: remove debug code
    # For now you can use this to visualize a single frame
    # show_point_cloud(xyz, rgb / 255, orig=np.zeros(3))
    self.voxel_map.add(
        camera_pose,
        xyz,
        rgb,
        depth=depth,
        base_pose=base_pose,
        obs=obs,
        K=self.robot.head._ros_client.rgb_cam.K,
    )
    if visualize_map:
        # Now draw 2d
        self.voxel_map.get_2d_map(debug=True)

def get_2d_map(self):
    """Get 2d obstacle map for low level motion planning and frontier-based exploration"""
    return self.voxel_map.get_2d_map()

def show(self, backend='open3d', **backend_kwargs) -> Tuple[np.ndarray, np.ndarray]:
    """Display the aggregated point cloud."""
    if backend == 'open3d':
        self._show_open3d(self, **backend_kwargs)
    elif backend == 'pytorch3d':
        self._show_pytorch3d(self, **backend_kwargs)
    else:
        raise NotImplementedError(f"Uknown backend {backend}, must be 'open3d' or 'pytorch3d")

def _show_pytorch3d(self, **plot_scene_kwargs):
    from bboxes_3d_plotly import plot_scenes_with_bboxes

    pc_xyz, pc_rgb = self.voxel_map.get_data()
    pcl = Pointclouds(points=pc_xyz, features=pc_rgb) # if # channels is 3, then interpret as RGB with in [0-1]
    
    bounds_np = np.stack([instance_view.bounds for instance_view in self.voxel_map._instance_views], axis=0)
    bounds = torch.tensor(bounds_np)
    instance_bboxs = BBoxes3D(
        boxes = bounds_np,
        features = [features_cat], # if # channels is 3, then interpret as RGB with in [0-1]
    )
    
    _default_plot_args = dict
        xaxis={"backgroundcolor":"rgb(200, 200, 230)"},
        yaxis={"backgroundcolor":"rgb(230, 200, 200)"},
        zaxis={"backgroundcolor":"rgb(200, 230, 200)"}, 
        axis_args=AxisArgs(showgrid=True),
        pointcloud_marker_size=3,
        pointcloud_max_points=200_000
    )
    fig = plot_scene_with_bboxes(
        plots = dict(
            f"Global scene": { 
                "Points": pcl, 
                "Instances": instance_bboxs
                # "cameras": cameras,
            },
            # Could add keyframes or instances here.
        ),
        **update(_default_plot_args, **plot_scene_kwargs)
    )
    
def _show_open3d(self):
    import open3d
    # Create a combined point cloud
    # Do the other stuff we need
    pc_xyz, pc_rgb = self.voxel_map.get_data()
    # TODO: easy version, just pt clouds
    # show_point_cloud(pc_xyz, pc_rgb / 255, orig=np.zeros(3))

    pcd = numpy_to_pcd(pc_xyz, pc_rgb / 255.0)
    geoms = create_visualization_geometries(pcd=pcd, orig=np.zeros(3))
    for instance_view in self.voxel_map._instance_views:
        mins, maxs = instance_view.bounds
        width, height, depth = maxs - mins

        # Create a mesh to visualzie where the instances were seen
        mesh_box = open3d.geometry.TriangleMesh.create_box(
            width=width, height=height, depth=depth
        )

        # Get vertex array from the mesh
        vertices = np.asarray(mesh_box.vertices)

        # Translate the vertices to the desired position
        vertices += mins
        triangles = np.asarray(mesh_box.triangles)

        # Create a wireframe mesh
        lines = []
        for tri in triangles:
            lines.append([tri[0], tri[1]])
            lines.append([tri[1], tri[2]])
            lines.append([tri[2], tri[0]])

        color = [1.0, 0.0, 0.0]  # Red color (R, G, B)
        colors = [color for _ in range(len(lines))]
        wireframe = open3d.geometry.LineSet(
            points=open3d.utility.Vector3dVector(vertices),
            lines=open3d.utility.Vector2iVector(lines),
        )
        # Get the colors and add to wireframe
        wireframe.colors = open3d.utility.Vector3dVector(colors)
        geoms.append(wireframe)

    # Show the geometries of where we have explored
    open3d.visualization.draw_geometries(geoms)
    return pc_xyz, pc_rgb

In [ ]:
rep = ObjectPointcloudRepresentation()
for i in tqdm(range(len(result['images']))):
    rep.step(
        result['images'][i], 
        result['depths'][i], 
        result['poses'][i], 
        result['intrinsics'][i], 
    )